In [ ]:
test_file = "../input/foursquare-location-matching/test.csv"
sub_file = "../input/foursquare-location-matching/sample_submission.csv"

In [ ]:
import pandas as pd
import numpy as np
from haversine import haversine
from unidecode import unidecode

In [ ]:
test_df = pd.read_csv(test_file)

In [ ]:
def pairwise_comp(lhs, rhs):
    """
    return True if 2 location is the same POI.
    """ 
    lhs_pos = (lhs["latitude"], lhs["longitude"])
    rhs_pos = (rhs["latitude"], rhs["longitude"])
    dist = haversine(lhs_pos, rhs_pos)
    
    if str(lhs['name']) == 'nan' or str(rhs['name']) == 'nan':
        names_insec = 0
    else:
        lhs_names = set(unidecode(lhs['name']).split())
        rhs_names = set(unidecode(rhs['name']).split())
        names_insec = len(lhs_names.intersection(rhs_names))
      
    if str(lhs['categories']) == 'nan' or str(rhs['categories']) == 'nan':
        cate_insec = 0
    else:
        lhs_cate = set(unidecode(lhs['categories']).split())
        rhs_cate = set(unidecode(rhs['categories']).split())
        cate_insec = len(lhs_cate.intersection(rhs_cate))

    if dist < 1 and (names_insec > 2 or cate_insec > 1):
        return True
    else:
        return False

In [ ]:
# Generate matches set.
match_sets = list()

def apply_fn(row1):
    in_set = False
    for match_set in match_sets:
        row2_id = next(iter(match_set))
        row2 = test_df[test_df["id"] == row2_id].iloc[0] 
        if pairwise_comp(row1, row2):
            match_set.add(row1["id"])
            in_set = True
            break
        
    if not in_set:
        match_sets.append({row1["id"]})
    

# Apply function to each row.
test_df.apply(apply_fn, axis=1)
match_sets

In [ ]:
# Match set to match list.
matches_list = dict()
for match_set in match_sets:
    for elem in match_set:
        matches_list[elem] = list(match_set)
        
matches_list

In [ ]:
# Init submission format from sample_submission.csv
sub_df = pd.read_csv(sub_file)
sub_df["matches"] = np.nan

# Set matches for each id.
def set_setter(selected_id: str):
    try:
        return " ".join(matches_list[selected_id])
    except:
        return ""
sub_df["matches"] = sub_df["id"].apply(lambda x: set_setter(x))

# Adding selfmatch.
#sub_df["matches"] = sub_df["id"] + " " + sub_df["matches"]

sub_df.head()

In [ ]:
sub_df.to_csv("submission.csv", index=False)